## 配置文件 (config.yaml)

```
# config.yaml
samples:
    A: data/samples/A.fastq
    B: data/samples/B.fastq
    C: data/samples/C.fastq
```

```
# Snakefile
def get_bwa_map_input_fastqs(wildcards):
    return config["samples"][wildcards.sample]
rule bwa_map:
    input:
        "data/genome.fa",
        get_bwa_map_input_fastqs
        # "data/samples/{sample}.fastq"
    output:
        "mapped_reads/{sample}.bam"
    params:
        rg=r"@RG\tID:{sample}\tSM:{sample}"
    log:
        "logs/bwa_mem/{sample}.log"
    threads: 8
    shell:
        "(bwa mem -R '{params.rg}' -t {threads} {input} | "
        "samtools view -Sb - > {output}) 2> {log}"


configfile: "config.yaml" # config file 的路徑
	/..more../
rule bcftools_call:
    input:
        fa="data/genome.fa",
        bam=expand("sorted_reads/{sample}.sorted.bam", sample=config["samples"]),
        bai=expand("sorted_reads/{sample}.sorted.bam.bai", sample=config["samples"])
/..more../
```
> + "config["samples"][wildcards.sample]"取得 config 檔案裡sample下的路徑，在shell中使用{input}時會將三個路徑都當輸入跑一遍  
>  
> + {sample} 是通配符，會得到樣本名稱(**不是路徑**，所以```logs/bwa_mem/```裡面是A,B,C的log檔)
>  
> + ```bam=expand("sorted_reads/{sample}.sorted.bam", sample=config["samples"])```會讓 bam = sorted_reads/{A,B,C}.sorted.bam 三個檔案，在shell用 {input.bam} 會三個檔案都跑
>  